In [1]:
import pandas as pd
import numpy as np

In [2]:
train=np.array(pd.read_csv("w2v_train_128.csv")).tolist()
test=np.array(pd.read_csv("w2v_test_128.csv")).tolist()

In [3]:
target=pd.read_csv("train_all.csv")

In [4]:
age_target=np.array(target["age"])
gender_target=np.array(target["gender"])-1

In [5]:
!pip install lightgbm

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 1.2 MB 868 kB/s eta 0:00:01


In [6]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(train,gender_target,test_size=0.04)
 
 
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
    'num_leaves': 5,
    'max_depth': 6,
    'min_data_in_leaf': 450,
    'learning_rate': 0.3,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.95,
    'bagging_freq': 5,
    'lambda_l1': 1,  
    'lambda_l2': 0.001,  # 越小l2正则程度越高
    'min_gain_to_split': 0.2,
    'verbose': 5,
    'is_unbalance': True
}
 
# train
print('Start training...')
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)
 
print('Start predicting...')
 
preds = gbm.predict(X_test, num_iteration=gbm.best_iteration)  # 输出的是概率结果
 
# 导出结果
threshold = 0.5
gender_pred=[]
for pred in preds:
    result = 1 if pred > threshold else 0
    gender_pred.append(result)
# 导出特征重要性
# importance = gbm.feature_importance()
# names = gbm.feature_name()
# with open('./feature_importance.txt', 'w+') as file:
#     for index, im in enumerate(importance):
#         string = names[index] + ', ' + str(im) + '\n'
#         file.write(string)

Start training...
[1]	valid_0's auc: 0.813943	valid_0's binary_logloss: 0.555095
Training until validation scores don't improve for 10 rounds
[2]	valid_0's auc: 0.879825	valid_0's binary_logloss: 0.498979
[3]	valid_0's auc: 0.895471	valid_0's binary_logloss: 0.466255
[4]	valid_0's auc: 0.904216	valid_0's binary_logloss: 0.442959
[5]	valid_0's auc: 0.911527	valid_0's binary_logloss: 0.425307
[6]	valid_0's auc: 0.915913	valid_0's binary_logloss: 0.408928
[7]	valid_0's auc: 0.920868	valid_0's binary_logloss: 0.395415
[8]	valid_0's auc: 0.925243	valid_0's binary_logloss: 0.382843
[9]	valid_0's auc: 0.927802	valid_0's binary_logloss: 0.371068
[10]	valid_0's auc: 0.930857	valid_0's binary_logloss: 0.362331
[11]	valid_0's auc: 0.934064	valid_0's binary_logloss: 0.35378
[12]	valid_0's auc: 0.936235	valid_0's binary_logloss: 0.346808
[13]	valid_0's auc: 0.937935	valid_0's binary_logloss: 0.340207
[14]	valid_0's auc: 0.939633	valid_0's binary_logloss: 0.334119
[15]	valid_0's auc: 0.940781	valid_

In [7]:
t=0
for i in range(0,len(y_test)):
    if gender_pred[i]==y_test[i]:
        t+=1
acct=t/len(gender_pred)
print(acct)#gender模型准确率

0.9180277777777778


In [8]:
print('Start predicting...')
 
preds = gbm.predict(test, num_iteration=gbm.best_iteration)  # 输出的是概率结果
 
# 导出结果
threshold = 0.5
gender_pred=[]
for pred in preds:
    result = 2 if pred > threshold else 1
    gender_pred.append(result)#预测结果

Start predicting...


In [9]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
 
X_train,X_test,y_train,y_test=train_test_split(train,age_target,test_size=0.04)
 
 
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 11,
    'metric': 'multi_error',
    'num_leaves': 300,
    'min_data_in_leaf': 100,
    'learning_rate': 0.2,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'lambda_l1': 0.4,
    'lambda_l2': 0.5,
    'min_gain_to_split': 0.2,
    'verbose': 5,
    'is_unbalance': True
}
 
# train
print('Start training...')
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)
 
print('Start predicting...')
 
preds = gbm.predict(X_test, num_iteration=gbm.best_iteration)  # 输出的是概率结果
 
# 导出结果
age_pred=[]
for pred in preds:
    result = prediction = int(np.argmax(pred))
    age_pred.append(result)
# 导出特征重要性
# importance = gbm.feature_importance()
# names = gbm.feature_name()
# with open('./feature_importance.txt', 'w+') as file:
#     for index, im in enumerate(importance):
#         string = names[index] + ', ' + str(im) + '\n'
#         file.write(string)

Start training...
[1]	valid_0's multi_error: 0.765694
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_error: 0.748833
[3]	valid_0's multi_error: 0.727778
[4]	valid_0's multi_error: 0.711694
[5]	valid_0's multi_error: 0.699056
[6]	valid_0's multi_error: 0.690944
[7]	valid_0's multi_error: 0.684472
[8]	valid_0's multi_error: 0.679333
[9]	valid_0's multi_error: 0.675306
[10]	valid_0's multi_error: 0.672694
[11]	valid_0's multi_error: 0.669583
[12]	valid_0's multi_error: 0.666611
[13]	valid_0's multi_error: 0.665083
[14]	valid_0's multi_error: 0.663056
[15]	valid_0's multi_error: 0.661111
[16]	valid_0's multi_error: 0.659889
[17]	valid_0's multi_error: 0.659528
[18]	valid_0's multi_error: 0.657778
[19]	valid_0's multi_error: 0.656556
[20]	valid_0's multi_error: 0.654444
[21]	valid_0's multi_error: 0.654583
[22]	valid_0's multi_error: 0.652083
[23]	valid_0's multi_error: 0.651806
[24]	valid_0's multi_error: 0.649583
[25]	valid_0's multi_error: 0.650028
[26]	

In [10]:
t=0
for i in range(0,len(y_test)):
    if age_pred[i]==y_test[i]:
        t+=1
acct=t/len(age_pred)
print(acct)#age模型准确率

0.3668888888888889


In [11]:
print('Start predicting...')
 
preds = gbm.predict(test, num_iteration=gbm.best_iteration)  # 输出的是概率结果
 
 
# 导出结果
age_pred=[]
for pred in preds:
    result = prediction = int(np.argmax(pred))
    age_pred.append(result)

Start predicting...


In [12]:
submission=pd.DataFrame(columns=("user_id","predicted_age","predicted_gender"))

In [13]:
test_all=pd.read_csv("test_all.csv")

In [14]:
submission["user_id"]=test_all["user_id"]
submission["predicted_age"]=age_pred
submission["predicted_gender"]=gender_pred

In [15]:
submission.to_csv("submission_6 1_size_128(new model).csv",index=False)

In [16]:
submission

,user_id,predicted_age,predicted_gender
0,3000001,3,1
1,3000002,6,2
2,3000003,2,2
3,3000004,3,1
4,3000005,3,1
...,...,...,...
999995,3999996,3,1
999996,3999997,2,1
999997,3999998,2,1
999998,3999999,2,1
